## 1. Import Required Libraries

In [ ]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Core ML libraries
from ultralytics import YOLO # pyright: ignore[reportPrivateImportUsage]
import torch
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Utilities
from pathlib import Path
import yaml
import json
from datetime import datetime

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

# Check GPU
print(f"PyTorch Version: {torch.__version__}")
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Load and Prepare Dataset

In [ ]:
# Define paths
DATA_DIR = Path('data')
DETECTION_DATA = DATA_DIR / 'detection' / 'dataset.yaml'
CLASSIFICATION_DATA = DATA_DIR / 'classification'
MODELS_DIR = Path('models')

# Create directories if they don't exist
MODELS_DIR.mkdir(exist_ok=True)
(MODELS_DIR / 'detector').mkdir(exist_ok=True)
(MODELS_DIR / 'classifier').mkdir(exist_ok=True)

print("Dataset Paths:")
print(f"  Detection: {DETECTION_DATA}")
print(f"  Classification: {CLASSIFICATION_DATA}")

# Load detection dataset config
if DETECTION_DATA.exists():
    with open(DETECTION_DATA, 'r') as f:
        det_config = yaml.safe_load(f)
    print(f"\nDetection Dataset: {det_config.get('nc')} class(es)")
else:
    print("⚠ Detection dataset not found. Prepare data first.")

if CLASSIFICATION_DATA.exists():
    train_dir = CLASSIFICATION_DATA / 'train'
    classes = [d.name for d in train_dir.iterdir() if d.is_dir()]
    print(f"\nClassification Classes: {classes}")
else:
    print("⚠ Classification dataset not found. Prepare data first.")

## 3. Initialize YOLOv8 Model

In [ ]:
# Initialize detection model (nano for edge, small for accuracy)
print("Initializing YOLOv8 Detection Model...")
detector = YOLO('yolov8n.pt')  # nano model

# Initialize classification model
print("Initializing YOLOv8 Classification Model...")
classifier = YOLO('yolov8n-cls.pt')  # nano classifier

# Set device
device = 0 if torch.cuda.is_available() else 'cpu'
print(f"\nUsing device: {'GPU (CUDA)' if device == 0 else 'CPU'}")

## 4. Configure Training Parameters

In [ ]:
# Load configurations
with open('configs/detector_config.yaml', 'r') as f:
    detector_config = yaml.safe_load(f)

with open('configs/classifier_config.yaml', 'r') as f:
    classifier_config = yaml.safe_load(f)

print("Detection Model Configuration:")
print(f"  Model Size: {detector_config['model_size']}")
print(f"  Epochs: {detector_config['epochs']}")
print(f"  Batch Size: {detector_config['batch_size']}")
print(f"  Image Size: {detector_config['img_size']}")
print(f"  Learning Rate: {detector_config['lr0']}")

print("\nClassification Model Configuration:")
print(f"  Model Size: {classifier_config['model_size']}")
print(f"  Epochs: {classifier_config['epochs']}")
print(f"  Batch Size: {classifier_config['batch_size']}")
print(f"  Image Size: {classifier_config['img_size']}")
print(f"  Learning Rate: {classifier_config['lr0']}")

## 5. Train Object Detection Model

In [ ]:
print("Starting Detection Model Training...\n")

# Train detector
detection_results = detector.train(
    data=str(DETECTION_DATA),
    epochs=detector_config['epochs'],
    imgsz=detector_config['img_size'],
    batch=detector_config['batch_size'],
    patience=detector_config['patience'],
    device=device,
    project=str(MODELS_DIR / 'detector'),
    name='flower_detector',
    exist_ok=True,
    
    # Optimizer and learning rate
    optimizer=detector_config['optimizer'],
    lr0=detector_config['lr0'],
    lrf=detector_config['lrf'],
    momentum=detector_config['momentum'],
    weight_decay=detector_config['weight_decay'],
    
    # Augmentation
    mosaic=detector_config['mosaic'],
    mixup=detector_config['mixup'],
    flipud=detector_config['flipud'],
    fliplr=detector_config['fliplr'],
    
    # Validation
    val=True,
    save=True,
    plots=True,
    verbose=True,
)

print("\n✓ Detection Training Completed!")

## 6. Train Classification Model

In [ ]:
print("Starting Classification Model Training...\n")

# Train classifier
classification_results = classifier.train(
    data=str(CLASSIFICATION_DATA),
    epochs=classifier_config['epochs'],
    imgsz=classifier_config['img_size'],
    batch=classifier_config['batch_size'],
    patience=classifier_config['patience'],
    device=device,
    project=str(MODELS_DIR / 'classifier'),
    name='flower_classifier',
    exist_ok=True,
    
    # Optimizer and learning rate
    optimizer=classifier_config['optimizer'],
    lr0=classifier_config['lr0'],
    lrf=classifier_config['lrf'],
    weight_decay=classifier_config['weight_decay'],
    
    # Augmentation
    flipud=classifier_config['flipud'],
    fliplr=classifier_config['fliplr'],
    degrees=classifier_config['degrees'],
    
    # Validation
    val=True,
    save=True,
    plots=True,
    verbose=True,
)

print("\n✓ Classification Training Completed!")

## 7. Evaluate Model Performance

In [ ]:
# Load best trained models
best_detector = YOLO(MODELS_DIR / 'detector' / 'flower_detector' / 'weights' / 'best.pt')
best_classifier = YOLO(MODELS_DIR / 'classifier' / 'flower_classifier' / 'weights' / 'best.pt')

# Validate detector
print("Validating Detection Model...")
det_metrics = best_detector.val(data=str(DETECTION_DATA))

print(f"\nDetection Model Metrics:")
print(f"  mAP@0.5: {det_metrics.box.map50:.4f}")
print(f"  mAP@0.5-0.95: {det_metrics.box.map:.4f}")
print(f"  Precision: {det_metrics.box.mp:.4f}")
print(f"  Recall: {det_metrics.box.mr:.4f}")

# Validate classifier
print("\nValidating Classification Model...")
clf_metrics = best_classifier.val(data=str(CLASSIFICATION_DATA))

print(f"\nClassification Model Metrics:")
print(f"  Top-1 Accuracy: {clf_metrics.top1:.4f}")
print(f"  Top-5 Accuracy: {clf_metrics.top5:.4f}")

## 8. Save and Export Trained Models

In [ ]:
# Create export directory
export_dir = MODELS_DIR / 'exported'
export_dir.mkdir(exist_ok=True)

print("Exporting models for edge deployment...\n")

# Export to ONNX (universal format)
print("Exporting to ONNX...")
try:
    best_detector.export(format='onnx', imgsz=640, device=device)
    print("  ✓ Detection model exported to ONNX")
except Exception as e:
    print(f"  ✗ ONNX export failed: {e}")

try:
    best_classifier.export(format='onnx', imgsz=224, device=device)
    print("  ✓ Classification model exported to ONNX")
except Exception as e:
    print(f"  ✗ ONNX export failed: {e}")

# Export to TFLite for mobile
print("\nExporting to TFLite...")
try:
    best_detector.export(format='tflite', imgsz=640, device='cpu')
    print("  ✓ Detection model exported to TFLite")
except Exception as e:
    print(f"  ✗ TFLite export skipped: {e}")

print("\n✓ Model export completed!")
print(f"Models saved to: {export_dir}")